<a href="https://colab.research.google.com/github/vinismachadoo/web_scrapping_farm/blob/main/catalogo_farm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests, threading
from multiprocessing.pool import ThreadPool, Pool
from bs4 import BeautifulSoup
import pandas as pd

In [20]:
url_base = 'https://www.farmrio.com.br'
subcategorias_farm = []

pagina_inicial = requests.get(url_base)
soup = BeautifulSoup(pagina_inicial.content, 'html.parser')
menu = soup.find_all('a', {'class':'menu__link menu__link--level-3'})
for m in menu:
    if 'http' not in m.get('href'):
        subcategorias_farm.append(url_base+m.get('href'))
    else:
        subcategorias_farm.append(m.get('href'))

print(len(subcategorias_farm))

79


In [22]:
def encontra_url_categorias(subcategoria):
    pagina_categoria = requests.get(subcategoria)
    soup = BeautifulSoup(pagina_categoria.text, 'html.parser')
    scripts = soup.find_all('script')
    for script in scripts:
        s = script.string
        if s is not None and 'var pagecount' in s:
            cat_url = s.split("load('")[1].split("' + page")[0]
            lista_url_cartegorias.append(url_base+cat_url)
        else:
            pass

lista_url_cartegorias = []
ThreadPool(15).map(encontra_url_categorias, subcategorias_farm)
urls_farm = list(dict.fromkeys(lista_url_cartegorias))
print(len(urls_farm))

35
